In [1]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import time
from datetime import datetime

In [2]:
df = pd.read_csv('files/Kaggle Competition/SentimentAnalysis/train.tsv', sep='\t')

In [3]:
df

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [4]:
#df.set_index('PhraseId', inplace=True)

In [5]:
df.shape

(156060, 4)

In [6]:
pd.set_option('display.max_rows', 85)

In [7]:
df.loc[1, 'Phrase']

'A series of escapades demonstrating the adage that what is good for the goose'

In [8]:
df.drop(columns=['PhraseId','SentenceId'], inplace=True)

In [9]:
df

,Phrase,Sentiment
0,A series of escapades demonstrating the adage ...,1
1,A series of escapades demonstrating the adage ...,2
2,A series,2
3,A,2
4,series,2
...,...,...
156055,Hearst 's,2
156056,forced avuncular chortles,1
156057,avuncular chortles,3
156058,avuncular,2


In [10]:
df_test.drop(columns=['PhraseId','SentenceId'], inplace=True)

In [11]:
start = datetime.now()
corpus = []
for i in range(0, 100000):
    review = re.sub('[^a-zA-Z]', ' ', df['Phrase'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)
end = datetime.now()
print(end - start)

0:03:02.559871


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()
y = df.iloc[:100000, 1].values

In [13]:
len(cv.get_feature_names())

5000

In [14]:
len(X)

100000

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [17]:
start = datetime.now()
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)                             

# Predicting the Test set results
y_pred = model.predict(X_test)
end = datetime.now()
print(end - start)

0:00:26.393403


In [18]:
model.score(X_test, y_test)

0.14455

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.14455

In [23]:
start = datetime.now()
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)                             

# Predicting th e Test set results
y_pred = model.predict(X_test)
end = datetime.now()
print(end - start)

1:16:11.367311


In [24]:
model.score(X_test, y_test)

0.63695

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.63695

In [26]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))
pickle.dump(cv, open('sparser.pkl', 'wb'))

In [28]:
df.Sentiment.unique()

array([1, 2, 3, 4, 0], dtype=int64)